
![alt text](https://i.ibb.co/3fWYYMv/banner-03-smaller.jpg)

*This notebook relies on Python, Google Cloud Platform and BigQuery

# Import libraries + Google Cloud Platform authentication

In [0]:
# Import libraries 
import pandas as pd
import numpy as np
import socket
import json

#Connect Python to your GCP project
from google.colab import auth
auth.authenticate_user()

# Connect to Google Cloud Storage (GCS) & BigQuery

## Enter your GCP project ID & bucket location
For your bucket location, go to GCP -> storage -> browser -> check location in table



In [0]:
# Import BigQuery from google.cloud
from google.cloud import bigquery

## Inputs from forms ##

GCP_PROJECT_ID = 'learned-spider-266010' #@param {type:"string"}

GCS_Bucket_Location = 'europe-west2' #@param ["asia-east1",	"asia-east2",	"asia-northeast1",	"asia-northeast2",	"asia-northeast3",	"asia-south1",	"asia-southeast1",	"australia-southeast1",	"europe-north1",	"europe-west1",	"europe-west2",	"europe-west3",	"europe-west4",	"europe-west6",	"northamerica-northeast1",	"southamerica-east1",	"us-central1",	"us-east1",	"us-east4",	"us-west1",	"us-west2"]

# Create a client instance for your project
client = bigquery.Client(project=GCP_PROJECT_ID, location=GCS_Bucket_Location)


## Add your GCS bucket name and file path to query
*   For your bucket name, go to GCP -> storage -> browser -> check 'name' in table

*   wildcard (*) must be used to query several files at once





In [0]:
GCS_Bucket_Name = 'apache_seo_logs' #@param {type:"string"}

GCS_Folder_Path = 'Loggy*' #@param {type:"string"}

GCS_Full_Path = "gs://" + GCS_Bucket_Name + "/" + GCS_Folder_Path

print("the path to your GCS folder is: " + GCS_Full_Path)

the path to your GCS folder is: gs://apache_seo_logs/Loggy*


## Apply SQL filtering

All the files in your bucket's folder path will be merged into a Bigquery table:

*   First, give the table a name (1st form field)
*   Then you can apply 2 SQL filters -> an 'open' one and a Search Engine selector 
*   Feel free to create/add your own filters!









In [0]:
## Form elements ##

table_name = 'log_sample' #@param {type:"string"}
SQL_1st_Filter ='ch.loggy' #@param {type:"string"}
SQL_Useragent_Filter = "Googlebot/2.1" #@param ["Googlebot/2.1", "YandexBot", "BingBot", "DuckDuckBot", "Baiduspider"] {allow-input: true}

# Temporary Bigquery table name
table_id = table_name

## Full SQL query ##

# Concatenate SQL filters above
SQLFilters = 'SELECT * FROM `{}` WHERE header LIKE "%' + SQL_1st_Filter + '%" AND header LIKE "%' + SQL_Useragent_Filter + '%"'
# Concatenate SQL filters above
sql = SQLFilters.format(table_id)

## Other config lines for BigQuery ##

# Configure the external data source and query job
external_config = bigquery.ExternalConfig("CSV")
external_config.source_uris = [GCS_Full_Path]
# 1st argument is where you ut the name of the header, here it is called 'header'                       
external_config.schema = [bigquery.SchemaField("header", "STRING"),]
# Should remain at 0 for default log file upload
external_config.options.skip_leading_rows = 0
# BigQuery job configuration
job_config = bigquery.QueryJobConfig(table_definitions={table_id: external_config})
# Auto-detect Schemas (hashed as not currently in use)
# job_config.autodetect = True

# Make an API request
query_job = client.query(sql, job_config=job_config)  
# Wait for the job to complete.
log_sample = list(query_job)

# Print SQL query sent to BigQuery
print('The SQL query sent to BigQuery is "' + SQLFilters + '"')

#log_sample

The SQL query sent to BigQuery is "SELECT * FROM `{}` WHERE header LIKE "%ch.loggy%" AND header LIKE "%Googlebot/2.1%""


Convert the list output to a Pandas dataframe. 



In [0]:
# Note that we convert a list of lists, not just a list.
df = pd.DataFrame.from_records(log_sample)

# Data Splitting/Cleansing

Split & cleanse each bit of useful from our cryptic logs, and create a column for each!

Name our unique column 'header'

In [0]:
df.rename(columns={0:'header'}, inplace=True)
df = df.iloc[1:]

Split IP addresses

In [0]:
# Split IP addresses
df[['IP','header']] = df["header"].str.split(" - - \[", 1, expand=True)
# Remove slash
df['IP'] = df['IP'].str.replace("Row\(\('","",n=1)

Split dates and times

In [0]:
# Split Date & Time
df[['Date_Time', 'header']] = df['header'].str.split("\] ", 1,
        expand=True)
df['Date_Time'] = df['Date_Time'].str.replace(':.*', '')

# Convert the Date_Time column from string to datetime format
df['Date'] = pd.to_datetime(df['Date_Time'], errors='coerce',
                            infer_datetime_format=True)

# Remove Time column, for clarity's sake
df = df.drop(['Date_Time'], axis=1)

# Check that the Date column has been converted to a time series
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2287 entries, 1 to 2287
Data columns (total 3 columns):
header    2287 non-null object
IP        2287 non-null object
Date      2287 non-null datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 53.7+ KB


Split domains

In [0]:
df[['Domain','header']] = df["header"].str.split(" \"", 1, expand=True)

Split methods (Get, Post etc...)

In [0]:
df[['Method','header']] = df["header"].str.split(" ", 1, expand=True)

Split URLs

In [0]:
df[['url','header']] = df["header"].str.split(" ", 1, expand=True)
# Remove slash (/) at the start of each URL
df['url'] = df['url'].str.replace('/','',n=1)

Split protocols

In [0]:
df[['httpProto','header']] = df["header"].str.split("\" ", 1, expand=True)

Split status codes

In [0]:
df[['httpCode','header']] = df["header"].str.split(" ", 1, expand=True)

Split 'time taken'


In [0]:
df[['timeTaken','header']] = df["header"].str.split(" ", 1, expand=True)

Split referral URLs


In [0]:
df[['urlRef','header']] = df["header"].str.split(" ", 1, expand=True)
df['urlRef'] = df['urlRef'].str.replace('\"', '')

Split User Agents


In [0]:
df[['userAgent','header']] = df["header"].str.split("\" ", 1, expand=True)

Split redirected URLs (when existing)

In [0]:
df[['remainder','header']] = df["header"].str.split('".*LOCATION=30.-', 1, expand=True)
df['header'] = df['header'].str.replace('\"', '')

Re-order columns

In [0]:
df = df[['IP','Date','Domain','Method','url','httpProto','httpCode','timeTaken','urlRef', 'userAgent','header']]
df.rename(columns={"header": "redirURL"}, inplace=True)

Check the output table, to ensure all is OK

In [0]:
df.head(2)

,IP,Date,Domain,Method,url,httpProto,httpCode,timeTaken,urlRef,userAgent,redirURL
1,66.249.65.161,2016-10-20,ch.loggy.com,GET,ch.loggy.com/FR/sacs-et-accessoires-femme,HTTP/1.1,200,50243,-,"""Mozilla/5.0 (compatible; Googlebot/2.1; +http...",None
2,66.249.65.171,2016-10-20,ch.loggy.com,GET,ch.loggy.com/IT/borse-accessori-donne,HTTP/1.1,200,49317,-,"""Mozilla/5.0 (compatible; Googlebot/2.1; +http...",None


# Create categorical columns

## Create a new column to categorise status codes

In [0]:
df['httpCodeClass'] = pd.np.where(df.httpCode.str.contains("^1.*"), 'Info (1XX)',
  pd.np.where(df.httpCode.str.contains("^2.*"), 'Success (2XX)',
    pd.np.where(df.httpCode.str.contains("^3.*"), 'Redirects (3XX)',
      pd.np.where(df.httpCode.str.contains("^4.*"), 'Client errors (4XX)', 'Server errors (5XX)'))))

## Create a column to categorise Search Engine Bots


In [0]:
df['SEBotClass'] = pd.np.where(df.userAgent.str.contains("YandexBot"), "YandexBot",
  pd.np.where(df.userAgent.str.contains("bingbot"), "BingBot",
    pd.np.where(df.userAgent.str.contains("DuckDuckBot"), "DuckDuckGo",
      pd.np.where(df.userAgent.str.contains("Baiduspider"), "Baidu",
        pd.np.where(df.userAgent.str.contains("Googlebot/2.1"), "GoogleBot", "Else")))))

Check the output table, to ensure all is OK

In [0]:
df.head(2)

,IP,Date,Domain,Method,url,httpProto,httpCode,timeTaken,urlRef,userAgent,redirURL,httpCodeClass,SEBotClass
1,66.249.65.161,2016-10-20,ch.loggy.com,GET,ch.loggy.com/FR/sacs-et-accessoires-femme,HTTP/1.1,200,50243,-,"""Mozilla/5.0 (compatible; Googlebot/2.1; +http...",None,Success (2XX),GoogleBot
2,66.249.65.171,2016-10-20,ch.loggy.com,GET,ch.loggy.com/IT/borse-accessori-donne,HTTP/1.1,200,49317,-,"""Mozilla/5.0 (compatible; Googlebot/2.1; +http...",None,Success (2XX),GoogleBot


# Spotting ‘spoofed’ Search Engine Bots


The function below verifies that IP addresses are genuinely from Googlebot.  The column 'isRealGbot?' will be created for that matter.

All credit due to [Tyler Reardon](https://twitter.com/) for that magic anti-spoofing detector! Initially available on [searchtools.io](https://www.searchtools.io/), we've simply integrated it in this notebook.

In [0]:
#@markdown By running this cell, all the IP addresses listed in our dataframe will be checked. (Double-click on this cell to unhide the code).

#@markdown ---

#Define the function

def reverse_dns(ip_address):
    '''
    This method returns the true host name for a 
    given IP address
    '''
    host_name = socket.gethostbyaddr(ip_address)
    reversed_dns = host_name[0]
    return reversed_dns

def forward_dns(reversed_dns):
    '''
    This method returns the first IP address string
    that responds as the given domain name
    '''
    try:
        data = socket.gethostbyname(reversed_dns)
        ip = str(data)
        return ip
    except Exception:
        print('error')
        return False

def ip_match(ip, true_ip):
    '''
    This method takes an ip address used for a reverse dns lookup
    and an ip address returned from a forward dns lookup
    and determines if they match.
    '''
    if ip == true_ip:
        ip_match = True
    else:
        ip_match = False
    return ip_match

def confirm_googlebot(host, ip_match):
    '''
    This method takes a hostname and the results of the ip_match() method
    and determines if an ip address from a log file is truly googlebot
    '''
    googlebot = False
    if host != False:
        if host.endswith('.googlebot.com') or host.endswith('.google.com'):
            if ip_match == True:
                #googlebot = 'Yes'
                googlebot = True
    return googlebot
                
def run(ip):
    try:
        host = reverse_dns(ip)
        true_ip = forward_dns(host)
        is_match = ip_match(ip, true_ip)
        return confirm_googlebot(host, is_match)
    except:
        #return 'No'
        return False

#Run the function against the IP addresses listed in the dataframe

df['isRealGbot?'] = df['IP'].apply(run)

Print the 'fake' IPs

In [0]:
df[df['isRealGbot?'] == False]

,IP,Date,Domain,Method,url,httpProto,httpCode,timeTaken,urlRef,userAgent,redirURL,httpCodeClass,SEBotClass,isRealGbot?


# Filter date & status codes, then export to csv!


## Filter by date

Check the Dataset time span ***pre-filtering***

In [0]:
print ('FYI, the dataset spans the following time intervals:')
print (df['Date'].min(), df['Date'].max())

FYI, the dataset spans the following time intervals:
2016-10-16 00:00:00 2016-10-20 00:00:00


Refine start and end points

In [0]:
start_date = '2016-10-16' #@param {type:"date"}
end_date = '2016-10-17' #@param {type:"date"}

print('start date for the exported csv is: ' + start_date)
print('end date for the exported csv is: ' + end_date)
print ('')

start date for the exported csv is: 2016-10-16
end date for the exported csv is: 2016-10-17



## Filter by status codes

Check status codes distribution

In [0]:
DfPivotCodes = df.groupby(['httpCode']).agg({'httpCode': ['count']})
DfPivotCodes.columns = ['_'.join(multi_index) for multi_index in DfPivotCodes.columns.ravel()]
DfPivotCodes = DfPivotCodes.reset_index()
DfPivotCodes

,httpCode,httpCode_count
0,200,2035
1,301,172
2,302,64
3,400,12
4,404,4


Select the http status codes you would like to export to csv 

In [0]:
myNewList = []

Code_2XX = True  # @param {type:"boolean"}
Code_3XX = True  # @param {type:"boolean"}
Code_4XX = True  # @param {type:"boolean"}
Code_5XX = True  # @param {type:"boolean"}

if Code_2XX == True:
    myNewList.append('Success (2XX)')
if Code_3XX == True:
    myNewList.append('Redirects (3XX)')
if Code_4XX == True:
    myNewList.append('Client errors (4XX)')
if Code_5XX == True:
    myNewList.append('Server errors (5XX)')

print (myNewList)

['Success (2XX)', 'Redirects (3XX)', 'Client errors (4XX)', 'Server errors (5XX)']


## Export your filtered dataset to csv

Type a name for your csv file

In [0]:
dfFiltered = df[(df['Date'] > start_date) & (df['Date'] <= end_date) & (df['httpCodeClass'].isin(myNewList))]

csvName = "csvExportNew" #@param {type:"string"}
csvName = csvName + '.csv'
dfFiltered.to_csv(csvName)
